## import

In [1]:
import transformers
import numpy as np
import pandas as pd
import re
import itertools

## 데이터 셋 준비 및 전처리

In [2]:
data_set = pd.read_csv("./train/train_drop_duplicates.csv")

In [3]:
from ast import literal_eval
def preprocessing(dataframe):
        subject_entity, subject_start, subject_end, subject_type = [], [], [], []
        object_entity, object_start, object_end, object_type = [], [], [], []

        for s, o in zip(dataframe["subject_entity"], dataframe["object_entity"]):
            s_dict = literal_eval(s)
            o_dict = literal_eval(o)

            subject_entity.append(s_dict["word"])
            subject_start.append(s_dict["start_idx"])
            subject_end.append(s_dict["end_idx"])
            subject_type.append(s_dict["type"])

            object_entity.append(o_dict["word"])
            object_start.append(o_dict["start_idx"])
            object_end.append(o_dict["end_idx"])
            object_type.append(o_dict["type"])
        
        entity_dataset = pd.DataFrame(
            {
                "subject_entity": subject_entity,
                "subject_start": subject_start,
                "subject_end": subject_end,
                "subject_type": subject_type,
                "object_entity": object_entity,
                "object_start": object_start,
                "object_end": object_end,
                "object_type": object_type,
            }
        )
        entity_dataset.reset_index(drop=True, inplace=True)
        all_dataset = dataframe[["id", "sentence", "label"]]
        all_dataset.reset_index(drop=True, inplace=True)
        preprocessing_dataframe = pd.concat([all_dataset, entity_dataset], axis=1)
        return preprocessing_dataframe

In [4]:
preprocessing_data_set = preprocessing(data_set)

In [5]:
index = ["sentence", "subject_entity", "object_entity"]
text_data_list=[preprocessing_data_set[i].to_list() for i in index]
text_data_list = list(itertools.chain(*text_data_list))
text_corpus = ' '.join(text_data_list)

## corpus 제너레이터 생성

In [6]:
training_corpus = (
    text_corpus[i : i + 1000]
    for i in range(0,len(text_corpus),1000)
)

## 토크나이저 학습

In [7]:
old_tokenizer = transformers.AutoTokenizer.from_pretrained("klue/roberta-large")

In [8]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

## 결과 확인

In [9]:
def old_token_encode_decode(sentence_list):
    for i in sentence_list:
        encode_result = old_tokenizer(i)["input_ids"]
        tokenized = old_tokenizer.tokenize(i)
        decode_result = old_tokenizer.decode(encode_result)
        print(f"{encode_result}  {tokenized}  {decode_result}")

In [10]:
def token_encode_decode(sentence_list):
    for i in sentence_list:
        encode_result = tokenizer(i)["input_ids"]
        tokenized = tokenizer.tokenize(i)
        decode_result = tokenizer.decode(encode_result)
        print(f"{encode_result}  {tokenized}  {decode_result}")

In [11]:
sentence_list = ["스티브 바라캇", "카게야마 슌", "리셴녠", "바즈겐 사르키샨", "데이비드 흄", "쥘 베른", "보루시아 묀헨글라트바흐","킵식스","핼리팩스", "데일리언 앳킨슨", 'ルールー・アムール']

In [12]:
token_encode_decode(sentence_list)

[0, 13303, 10742, 4750, 2]  ['스티브', '바라', '##캇']  [CLS] 스티브 바라캇 [SEP]
[0, 33322, 2783, 2]  ['카게야마', '슌']  [CLS] 카게야마 슌 [SEP]
[0, 23511, 2]  ['리셴녠']  [CLS] 리셴녠 [SEP]
[0, 2566, 3711, 4187, 36083, 2]  ['바', '##즈', '##겐', '사르키샨']  [CLS] 바즈겐 사르키샨 [SEP]
[0, 7830, 3537, 2]  ['데이비드', '흄']  [CLS] 데이비드 흄 [SEP]
[0, 3084, 15868, 2]  ['쥘', '베른']  [CLS] 쥘 베른 [SEP]
[0, 8509, 20462, 2]  ['보루시아', '묀헨글라트바흐']  [CLS] 보루시아 묀헨글라트바흐 [SEP]
[0, 3299, 14317, 2]  ['킵', '##식스']  [CLS] 킵식스 [SEP]
[0, 50894, 31891, 2]  ['핼리', '##팩스']  [CLS] 핼리팩스 [SEP]
[0, 18396, 3830, 31183, 4049, 2]  ['데일리', '##언', '앳킨', '##슨']  [CLS] 데일리언 앳킨슨 [SEP]
[0, 457, 4659, 4660, 4659, 460, 426, 5285, 4659, 4660, 2]  ['ル', '##ー', '##ル', '##ー', '・', 'ア', '##ム', '##ー', '##ル']  [CLS] ルールー ・ アムール [SEP]


In [119]:
old_token_encode_decode(sentence_list)

[0, 14004, 3, 2]  ['스티브', '[UNK]']  [CLS] 스티브 [UNK] [SEP]
[0, 1697, 2318, 12950, 3, 2]  ['카', '##게', '##야마', '[UNK]']  [CLS] 카게야마 [UNK] [SEP]
[0, 3, 2]  ['[UNK]']  [CLS] [UNK] [SEP]
[0, 1118, 2228, 2171, 3, 2]  ['바', '##즈', '##겐', '[UNK]']  [CLS] 바즈겐 [UNK] [SEP]
[0, 12145, 3, 2]  ['데이비드', '[UNK]']  [CLS] 데이비드 [UNK] [SEP]
[0, 3, 1148, 2202, 2]  ['[UNK]', '베', '##른']  [CLS] [UNK] 베른 [SEP]
[0, 23326, 3837, 3, 2]  ['보루', '##시아', '[UNK]']  [CLS] 보루시아 [UNK] [SEP]
[0, 3, 2]  ['[UNK]']  [CLS] [UNK] [SEP]
[0, 3, 2]  ['[UNK]']  [CLS] [UNK] [SEP]
[0, 7660, 2347, 3, 2]  ['데일리', '##언', '[UNK]']  [CLS] 데일리언 [UNK] [SEP]
[0, 3, 181, 3, 2]  ['[UNK]', '・', '[UNK]']  [CLS] [UNK] ・ [UNK] [SEP]


## 학습된 토크나이저 저장

In [14]:
tokenizer.save_pretrained("New_klue_token_list")

('New_klue_token_list/tokenizer_config.json',
 'New_klue_token_list/special_tokens_map.json',
 'New_klue_token_list/vocab.txt',
 'New_klue_token_list/added_tokens.json',
 'New_klue_token_list/tokenizer.json')

In [15]:
old_tokenizer.save_pretrained("Origin_klue_token_list")

('Origin_klue_token_list/tokenizer_config.json',
 'Origin_klue_token_list/special_tokens_map.json',
 'Origin_klue_token_list/vocab.txt',
 'Origin_klue_token_list/added_tokens.json',
 'Origin_klue_token_list/tokenizer.json')

In [16]:
with open("./New_klue_token_list/vocab.txt","r") as f:
    data = f.readlines()
data = [re.sub(r'\n','',i) for i in data]

## Train Set에서 얼마나 사용되는지 확인

In [17]:
tokens_list = []  # 실제 토큰
token_ids_list = []  # klue/roberta-large에서의 input_ids
used_len_list = [] # train셋에서 사용된 양

for idx, key in enumerate(data[89:3578]):
    key = re.sub("##","",key)
    token_ids = old_tokenizer(key, add_special_tokens=False)['input_ids'][0]
    used = len(re.findall(key,text_corpus))
    tokens_list.append(key)
    token_ids_list.append(token_ids)
    used_len_list.append(used)

In [18]:
df = pd.DataFrame()
df['token'] =tokens_list
df['tokenize'] = token_ids_list
df['used_in_train'] = used_len_list

In [19]:
df

,token,tokenize,used_in_train
0,{,94,3
1,|,95,3579265
2,},96,3
3,~,97,3184
4,£,3,7
...,...,...,...
3484,･,3,11
3485,𑀕,3,1
3486,𑀥,3,1
3487,𑀫,3,1


In [20]:
df.to_csv("./unknown_token.csv")

## 후보군 탐색

In [21]:
print(data.index('Á'), data.index('ʒ'))

101 161


In [22]:
print(data.index('ΐ'), data.index('ῶ'))

169 317


In [23]:
print(data.index('あ'), data.index('ー'))

378 461


In [24]:
print(data.index('㫕'), data.index('李'))

477 3559


In [25]:
Char_Total = data[data.index('Á'):data.index('ʒ')+1] + data[data.index('ΐ'):data.index('ῶ')+1] + data[data.index('あ'):data.index('ー')+1] + data[data.index('㫕'):data.index('李')+1]

In [26]:
len(Char_Total)

3377

In [804]:
Char_Total_candidate = []
for idx, key in enumerate(Char_Total):
    key = re.sub("##","",key)
    token_ids = old_tokenizer(key, add_special_tokens=False)['input_ids']
    used = len(re.findall(key,text_corpus))
    if 3 in token_ids:
        if used>=20:
            # print(f"{key}   {token_ids}   {used}")
            Char_Total_candidate.append(key)

In [805]:
len(Char_Total_candidate)

48

In [29]:
tokens_list = []  # 실제 토큰
token_ids_list = []  # klue/roberta-large에서의 input_ids
used_len_list = [] # train셋에서 사용된 양


for idx, key in enumerate(Char_Total_candidate):
    key = re.sub("##","",key)
    token_ids = old_tokenizer(key, add_special_tokens=False)['input_ids'][0]
    used = len(re.findall(key,text_corpus))
    tokens_list.append(key)
    token_ids_list.append(token_ids)
    used_len_list.append(used)

In [30]:
df = pd.DataFrame()
df['token'] =tokens_list
df['tokenize'] = token_ids_list
df['used_in_train'] = used_len_list

In [31]:
df

,token,tokenize,used_in_train
0,É,3,9
1,Ö,3,7
2,Ú,3,6
3,ß,3,5
4,à,3,9
...,...,...,...
426,헹,3,8
427,홋,3,108
428,훙,3,24
429,흄,3,7


In [32]:
df.to_csv("./add_tokens_candidate.csv")

## 업로드
- 교안 참고, 깃허브와 유사

In [40]:
object_entity_types = ["PER", "ORG", "LOC", "DAT", "POH", "NOH"]

In [41]:
for i in object_entity_types:
    print(old_tokenizer(i, add_special_tokens=False))

{'input_ids': [21639], 'token_type_ids': [0], 'attention_mask': [1]}
{'input_ids': [51, 2107, 2341], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}
{'input_ids': [24678, 2108], 'token_type_ids': [0, 0], 'attention_mask': [1, 1]}
{'input_ids': [27585, 2081], 'token_type_ids': [0, 0], 'attention_mask': [1, 1]}
{'input_ids': [12662, 2109], 'token_type_ids': [0, 0], 'attention_mask': [1, 1]}
{'input_ids': [18168, 2109], 'token_type_ids': [0, 0], 'attention_mask': [1, 1]}


In [42]:
entity_tokens = ["ORG", "LOC", "DAT", "POH", "NOH","SUBJ","OBJ"]  # PER을 제외한 토큰 추가

In [43]:
symbol_tokens = ['£', '☎', '€', '""']

In [44]:
for i in symbol_tokens:
    print(old_tokenizer(i))

{'input_ids': [0, 3, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}
{'input_ids': [0, 3, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}
{'input_ids': [0, 3, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}
{'input_ids': [0, 6, 6, 2], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}


In [45]:
sampling_count = 500-len(Char_Total_candidate + entity_tokens + symbol_tokens)

In [46]:
sampling_count

58

In [47]:
import random

random.seed(42)

In [48]:
Char_Total_candidate_four = []
for idx, key in enumerate(data[data.index('㫕'):data.index('李')+1]):
    key = re.sub("##","",key)
    token_ids = old_tokenizer(key, add_special_tokens=False)['input_ids']
    used = len(re.findall(key,text_corpus))
    if 3 in token_ids:
        if used==4:
            # print(f"{key}   {token_ids}   {used}")
            Char_Total_candidate_four.append(key)

In [49]:
Char_Total_candidate_four = random.sample(Char_Total_candidate_four, sampling_count)

In [50]:
len(Char_Total_candidate_four)

58

In [51]:
len(Char_Total_candidate + entity_tokens + symbol_tokens + Char_Total_candidate_four)

500

In [53]:
Total_Add_Tokens = Char_Total_candidate + entity_tokens + symbol_tokens + Char_Total_candidate_four

In [54]:
len(Total_Add_Tokens)

500

In [48]:
with open("./temp","w") as f:
    t= ["123","312"]
    for i in t:
        f.write(i+'\n')

In [55]:
with open("./Origin_klue_token_list/vocab.txt","r") as f:
    origin_vocab_list = f.readlines()

In [58]:
origin_vocab_list = [re.sub('\n','',i) for i in origin_vocab_list]

In [59]:
origin_vocab_list

['[CLS]',
 '[PAD]',
 '[SEP]',
 '[UNK]',
 '[MASK]',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '²',
 '´',
 '·',
 '×',
 'é',
 'а',
 'е',
 'и',
 'н',
 'о',
 'р',
 'с',
 'т',
 'ᆞ',
 '—',
 '―',
 '‘',
 '’',
 '“',
 '”',
 '•',
 '․',
 '‥',
 '…',
 '‧',
 '※',
 '℃',
 'ℓ',
 'Ⅰ',
 'Ⅱ',
 'Ⅲ',
 '↑',
 '→',
 '∙',
 '∼',
 '≪',
 '≫',
 '⊙',
 '①',
 '②',
 '③',
 '④',
 '⑤',
 '⑥',
 '⑦',
 '─',
 '━',
 '│',
 '■',
 '□',
 '▲',
 '△',
 '▶',
 '▷',
 '▼',
 '◆',
 '◇',
 '◈',
 '○',
 '●',
 '★',
 '☆',
 '☞',
 '♀',
 '♡',
 

In [62]:
re_Vocab = origin_vocab_list[:31500] + Total_Add_Tokens  # 순서 유의

In [63]:
len(re_Vocab)

32000

In [64]:
re_Vocab

['[CLS]',
 '[PAD]',
 '[SEP]',
 '[UNK]',
 '[MASK]',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '²',
 '´',
 '·',
 '×',
 'é',
 'а',
 'е',
 'и',
 'н',
 'о',
 'р',
 'с',
 'т',
 'ᆞ',
 '—',
 '―',
 '‘',
 '’',
 '“',
 '”',
 '•',
 '․',
 '‥',
 '…',
 '‧',
 '※',
 '℃',
 'ℓ',
 'Ⅰ',
 'Ⅱ',
 'Ⅲ',
 '↑',
 '→',
 '∙',
 '∼',
 '≪',
 '≫',
 '⊙',
 '①',
 '②',
 '③',
 '④',
 '⑤',
 '⑥',
 '⑦',
 '─',
 '━',
 '│',
 '■',
 '□',
 '▲',
 '△',
 '▶',
 '▷',
 '▼',
 '◆',
 '◇',
 '◈',
 '○',
 '●',
 '★',
 '☆',
 '☞',
 '♀',
 '♡',
 

In [65]:
origin_vocab_list == re_Vocab

False

In [66]:
origin_vocab_list[:31500] == re_Vocab[:35000]

False

In [67]:
for i in range(31500):
    if origin_vocab_list[i] != re_Vocab[i]:
        print(origin_vocab_list[i], re_Vocab[i])

In [68]:
with open("./Origin_klue_token_list/vocab.txt","w") as f:
    for i in re_Vocab:
        f.write(i+'\n')

In [75]:
import json

with open("./Origin_klue_token_list/tokenizer.json") as f:
    tokenizer_json = json.load(f)

In [80]:
tokenizer_json["model"]["vocab"]

{'[CLS]': 0,
 '[PAD]': 1,
 '[SEP]': 2,
 '[UNK]': 3,
 '[MASK]': 4,
 '!': 5,
 '"': 6,
 '#': 7,
 '$': 8,
 '%': 9,
 '&': 10,
 "'": 11,
 '(': 12,
 ')': 13,
 '*': 14,
 '+': 15,
 ',': 16,
 '-': 17,
 '.': 18,
 '/': 19,
 '0': 20,
 '1': 21,
 '2': 22,
 '3': 23,
 '4': 24,
 '5': 25,
 '6': 26,
 '7': 27,
 '8': 28,
 '9': 29,
 ':': 30,
 ';': 31,
 '<': 32,
 '=': 33,
 '>': 34,
 '?': 35,
 '@': 36,
 'A': 37,
 'B': 38,
 'C': 39,
 'D': 40,
 'E': 41,
 'F': 42,
 'G': 43,
 'H': 44,
 'I': 45,
 'J': 46,
 'K': 47,
 'L': 48,
 'M': 49,
 'N': 50,
 'O': 51,
 'P': 52,
 'Q': 53,
 'R': 54,
 'S': 55,
 'T': 56,
 'U': 57,
 'V': 58,
 'W': 59,
 'X': 60,
 'Y': 61,
 'Z': 62,
 '[': 63,
 ']': 64,
 '^': 65,
 '_': 66,
 '`': 67,
 'a': 68,
 'b': 69,
 'c': 70,
 'd': 71,
 'e': 72,
 'f': 73,
 'g': 74,
 'h': 75,
 'i': 76,
 'j': 77,
 'k': 78,
 'l': 79,
 'm': 80,
 'n': 81,
 'o': 82,
 'p': 83,
 'q': 84,
 'r': 85,
 's': 86,
 't': 87,
 'u': 88,
 'v': 89,
 'w': 90,
 'x': 91,
 'y': 92,
 'z': 93,
 '{': 94,
 '|': 95,
 '}': 96,
 '~': 97,
 '²': 98,

In [85]:
Vocab_dict = dict()

In [90]:
for idx,key in enumerate(re_Vocab):
    Vocab_dict[key]=idx

In [91]:
Vocab_dict

{'[CLS]': 0,
 '[PAD]': 1,
 '[SEP]': 2,
 '[UNK]': 3,
 '[MASK]': 4,
 '!': 5,
 '"': 6,
 '#': 7,
 '$': 8,
 '%': 9,
 '&': 10,
 "'": 11,
 '(': 12,
 ')': 13,
 '*': 14,
 '+': 15,
 ',': 16,
 '-': 17,
 '.': 18,
 '/': 19,
 '0': 20,
 '1': 21,
 '2': 22,
 '3': 23,
 '4': 24,
 '5': 25,
 '6': 26,
 '7': 27,
 '8': 28,
 '9': 29,
 ':': 30,
 ';': 31,
 '<': 32,
 '=': 33,
 '>': 34,
 '?': 35,
 '@': 36,
 'A': 37,
 'B': 38,
 'C': 39,
 'D': 40,
 'E': 41,
 'F': 42,
 'G': 43,
 'H': 44,
 'I': 45,
 'J': 46,
 'K': 47,
 'L': 48,
 'M': 49,
 'N': 50,
 'O': 51,
 'P': 52,
 'Q': 53,
 'R': 54,
 'S': 55,
 'T': 56,
 'U': 57,
 'V': 58,
 'W': 59,
 'X': 60,
 'Y': 61,
 'Z': 62,
 '[': 63,
 ']': 64,
 '^': 65,
 '_': 66,
 '`': 67,
 'a': 68,
 'b': 69,
 'c': 70,
 'd': 71,
 'e': 72,
 'f': 73,
 'g': 74,
 'h': 75,
 'i': 76,
 'j': 77,
 'k': 78,
 'l': 79,
 'm': 80,
 'n': 81,
 'o': 82,
 'p': 83,
 'q': 84,
 'r': 85,
 's': 86,
 't': 87,
 'u': 88,
 'v': 89,
 'w': 90,
 'x': 91,
 'y': 92,
 'z': 93,
 '{': 94,
 '|': 95,
 '}': 96,
 '~': 97,
 '²': 98,

In [93]:
tokenizer_json["model"]["vocab"] = Vocab_dict

In [95]:
tokenizer_json["model"]["vocab"]["☎"]

31939

In [97]:
with open("./Origin_klue_token_list/tokenizer.json","w") as f:
    json.dump(tokenizer_json, f)

- 생성된 파일 네개를 교안 참고하여 레포에 넣고 커밋 & 푸시

In [123]:
new_tokenizer = transformers.AutoTokenizer.from_pretrained("JunHyung1206/sajo_klue_roberta_large", do_basic_tokenize=False)

Downloading:   0%|          | 0.00/245k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/751k [00:00<?, ?B/s]

# 텍스트 전처리

## 따옴표 계열 처리

- """" 제거

In [388]:
def four_double_quotation_delete(sentence):
    sentence = re.sub('""""','',sentence)
    return sentence

- "" -> "으로 변경 (감싸져 있는 경우)

In [458]:
def double_quotation_to_quotation(sentence):
    sentence = re.sub(r'""','"',sentence)
    return sentence

- "" 계열은 ""로 통일, '계열은 '로 통일

In [390]:
def quotation_consist(sentence):
    sentence = re.sub(r'[‘’′＇`]', "'",sentence)
    sentence = re.sub(r'[“”]','"',sentence)
    return sentence

In [391]:
quotation_consist('`')

"'"

## 괄호 계열 통일

- 소괄호

In [392]:
def parenthesis_consist(sentence):
    sentence = re.sub(r'（','(',sentence)
    sentence = re.sub(r'）',')',sentence)
    return sentence

In [393]:
parenthesis_consist("（안녕하세요）")

'(안녕하세요)'

- 브래킷

In [394]:
def bracket_consist(sentence):
    sentence = re.sub(r'[«≪〈]','《',sentence)
    sentence = re.sub(r'[»≫〉]','》',sentence)
    return sentence

In [395]:
bracket_consist('«≪〈《»≫〉》')

'《《《《》》》》'

- 박스

In [396]:
def box_consist(sentence):
    sentence = re.sub(r'[˹『｢]','「',sentence)
    sentence = re.sub(r'[˼』｣]','」',sentence)
    return sentence

In [397]:
box_consist('˹『｢˼』｣')

'「「「」」」'

## - 통일

In [398]:
def bar_consist(sentence):
    sentence = re.sub(r'[–—―]','-',sentence)
    return sentence

In [399]:
bar_consist('–—―')

'---'

## 가운데 점 통일

In [400]:
def cdot_consist(sentence):
    sentence = re.sub(r'[⋅･•・]','·',sentence)
    return sentence

In [401]:
cdot_consist('⋅･・•·')

'·····'

## 의미상 필요없어 보이는 특수기호 제거

In [402]:
def symbol_delete(sentence):
    sentence = re.sub(r'[▲△▴▵□☎☏⁺∞Ⓐ®𑀫𑀕𑀥★☆♡♥※˘³𑀫𑀕𑀥]','',sentence)
    return sentence

## 야구에서 사용되는 분수표현 대체 (띄어쓰기 필요)

In [403]:
def frac_consist(sentence):
    sentence = re.sub(r'⅓',' 1/3',sentence)
    sentence = re.sub(r'⅔',' 2/3',sentence)
    return sentence

In [404]:
frac_consist('9⅔이닝')

'9 2/3이닝'

## 회사에서 사용되는 표현 통일

In [405]:
def company_consist(sentence):
    sentence = re.sub(r'㈔','(사)',sentence)
    sentence = re.sub(r'㈜','(주)',sentence)
    return sentence

## 단위 기호 통일

In [406]:
def measure_consist(sentence):
    sentence = re.sub(r'㎏','kg',sentence)
    sentence = re.sub(r'ℓ','L',sentence)
    sentence = re.sub(r'㎖','mL',sentence)
    sentence = re.sub(r'㎜','mm',sentence)
    sentence = re.sub(r'㎞','km',sentence)
    sentence = re.sub(r'㎡','m²',sentence)
    sentence = re.sub(r'㎿','MW',sentence)
    sentence = re.sub(r'ｍ','m',sentence)
    sentence = re.sub(r'°','도',sentence)
    sentence = re.sub(r'℃','도',sentence)
    
    return sentence

In [407]:
measure_consist('㎏ ℓ ㎖ ㎜ ㎞ ㎡  ㎿  ｍ')

'kg L mL mm km m²  MW  m'

## roma숫자 숫자로 변경

In [408]:
def roma_to_num(sentence):
    roma_Alpha_Upper=['Ⅰ','Ⅱ','Ⅲ','Ⅳ','Ⅴ','Ⅵ','Ⅶ','Ⅷ','Ⅸ','Ⅹ','Ⅺ','Ⅻ']
    roma_Alpha_Lower=['ⅰ','ⅱ','ⅲ','ⅳ','ⅴ','ⅵ','ⅶ','ⅷ','ⅸ','ⅹ','ⅺ','ⅻ']
    for i in range(0,12):
        sentence = re.sub(roma_Alpha_Upper[i],str(i+1),sentence)
        sentence = re.sub(roma_Alpha_Lower[i],str(i+1),sentence)
    return sentence

In [409]:
sentence = '"국제 농구 연맹은 1932년 스위스 제네바에서 ""국제 아마추어 농구 연맹""("""")이라는 명칭으로 창설되었다."'

## 그 외 유니코드 이슈

In [410]:
def unicode_err_consist(sentence):
    sentence = re.sub(r'％','%',sentence)
    sentence = re.sub(r'，',',',sentence)
    sentence = re.sub(r'／','/',sentence)
    sentence = re.sub(r'１','1',sentence)
    sentence = re.sub(r'：',':',sentence)
    sentence = re.sub(r'？','?',sentence)
    sentence = re.sub(r'～','~',sentence)
    
    return sentence

In [411]:
unicode_err_consist('"황보유의(皇甫兪義, ？ ~ 1042년)는 고려 전기의 문신이다."')

'"황보유의(皇甫兪義, ? ~ 1042년)는 고려 전기의 문신이다."'

## 전체 Text Preprocessing

In [459]:
def text_preprocessing(sentence):
    # 따옴표 계열 처리 과정
    sentence = four_double_quotation_delete(sentence)
    sentence = double_quotation_to_quotation(sentence)
    sentence = quotation_consist(sentence)
    
    # 괄호 계열 처리
    sentence = parenthesis_consist(sentence)
    sentence = bracket_consist(sentence)
    sentence = box_consist(sentence)
    
    # bar 처리
    sentence = bar_consist(sentence)
    
    # cdot 처리
    sentence = cdot_consist(sentence)
    
    # 특수 기호 제거
    sentence = symbol_delete(sentence)
    
    # 분수 표현(야구) 통일
    sentence = frac_consist(sentence)
    
    # 회사표현 통일
    sentence = company_consist(sentence)
    
    # 단위표현 통일
    sentence = measure_consist(sentence)
    
    # 로마숫자 숫자로 변경
    sentence = roma_to_num(sentence)
    
    # 그외 유니코드 이슈 처리
    sentence = unicode_err_consist(sentence)
    
    # 빈괄호 제거
    sentence = re.sub(r'()','',sentence)
    sentence = re.sub(r'《》','',sentence)
    
    # 공백 두번 이상인 것 처리 및 앞 뒤 공백 제거
    sentence = re.sub(r' +', ' ',sentence).strip()
    
    return sentence

In [460]:
sentence = '"1933년 1년 연하의 루이즈 웰스(Louise Wells)와 결혼하였으며 1936년 조니 라이트(Johnnie Wright)와 함께 컨트리 음악 보컬 듀오 """"의 보컬리스트 겸 기타리스트로 가수 첫 데뷔하였고 이듬해 1937년 컨트리 음악 보컬 그룹 ""앵글린 브라더스(The Anglin Brothers)""의 보컬리스트 겸 기타리스트로 활동하였으며 1938년 솔로 가수 데뷔하였다."'

In [461]:
text_preprocessing(sentence)

'"1933년 1년 연하의 루이즈 웰스(Louise Wells)와 결혼하였으며 1936년 조니 라이트(Johnnie Wright)와 함께 컨트리 음악 보컬 듀오 의 보컬리스트 겸 기타리스트로 가수 첫 데뷔하였고 이듬해 1937년 컨트리 음악 보컬 그룹 "앵글린 브라더스(The Anglin Brothers)"의 보컬리스트 겸 기타리스트로 활동하였으며 1938년 솔로 가수 데뷔하였다."'

In [462]:
sentence = '"바르셀로나 계약서에 역대 최고 이적료의 인수조항이 붙어 있었는데, 레알 마드리드가 인수 조항대로 지불해 피구의 마드리드 합류로 플로렌티노 페레스의 """"은하군단 시대""""의 막을 올려 매 년마다 세계구급의 선수가 입단하게 되었다."'

In [463]:
text_preprocessing(sentence)

'"바르셀로나 계약서에 역대 최고 이적료의 인수조항이 붙어 있었는데, 레알 마드리드가 인수 조항대로 지불해 피구의 마드리드 합류로 플로렌티노 페레스의 은하군단 시대의 막을 올려 매 년마다 세계구급의 선수가 입단하게 되었다."'

In [464]:
sentence = '류현진 선수는 올해 29경기에 선발 등판해 182⅔ 동안 평균자책점 2.32를 기록했다'

In [465]:
text_preprocessing(sentence)

'류현진 선수는 올해 29경기에 선발 등판해 182 2/3 동안 평균자책점 2.32를 기록했다'

In [468]:
sentence = '\"\"\"공식을 무시하지 마(Don\'t fuck with the formula)\"\"는 비치 보이스의 공동 창립자 마이크 러브가 1967년경 발언했다고 알려진 인용구다.\"'

In [469]:
text_preprocessing(sentence)

'""공식을 무시하지 마(Don\'t fuck with the formula)"는 비치 보이스의 공동 창립자 마이크 러브가 1967년경 발언했다고 알려진 인용구다."'

In [470]:
sentence = '"브래드 리틀(Brad Little, 1964년 4월 10일 ~)는 미국의 뮤지컬 배우이자 가수로, ""Phantom of the Opera"" 및 ""Jekyll and Hyde"" 의 주인공으로도 유명하다."'

In [471]:
text_preprocessing(sentence)

'"브래드 리틀(Brad Little, 1964년 4월 10일 ~)는 미국의 뮤지컬 배우이자 가수로, "Phantom of the Opera" 및 "Jekyll and Hyde" 의 주인공으로도 유명하다."'

In [472]:
sentence = '인천광역시교육청(도성훈 교육감)은 초•중•고 기초학력 담당 교사 520여명을 대상으로 2일(월)시교육청 대회의실에서 내년도부터 실시하는 ‘기초학력보장지원 사업 선택제’ 설명회를 실시하였다.'

In [473]:
text_preprocessing(sentence)

"인천광역시교육청(도성훈 교육감)은 초·중·고 기초학력 담당 교사 520여명을 대상으로 2일(월)시교육청 대회의실에서 내년도부터 실시하는 '기초학력보장지원 사업 선택제' 설명회를 실시하였다."

In [475]:
sentence = '전호환 부산대학교 총장은 “90년 간 기업을 지속해 온 대단함을 존경하며 지역과 함께하며 소비자들에게 기쁨을 전달해 준 것에 감사를 표한다”며 “경기침체로 기업환경이 어려움을 겪고 있을텐데 \'붕정만리\'의 뜻을 되새기며 앞날의 양양함과 원대한 사업이 이루어지도록 염원한다”고 무학 임직원들에게 마음을 전했다.'

In [476]:
text_preprocessing(sentence)

'전호환 부산대학교 총장은 "90년 간 기업을 지속해 온 대단함을 존경하며 지역과 함께하며 소비자들에게 기쁨을 전달해 준 것에 감사를 표한다"며 "경기침체로 기업환경이 어려움을 겪고 있을텐데 \'붕정만리\'의 뜻을 되새기며 앞날의 양양함과 원대한 사업이 이루어지도록 염원한다"고 무학 임직원들에게 마음을 전했다.'

In [477]:
sentence = '"안산시(시장 윤화섭)는 ㈔한국인터넷전문가협회가 주최하고 과학기술정보통신부·한국인터넷진흥원 등이 후원하는 ‘인터넷에코어워드(iEco Award) 2019’와 ‘소셜아이어워드(SOCIALl i-Award) 2019’에서 각각 ‘인터넷소통분야 대상’, ‘시군구 지자체 페이스북 분야 최우수상’을 수상했다고 4일 밝혔다."'

In [478]:
text_preprocessing(sentence)

'"안산시(시장 윤화섭)는 (사)한국인터넷전문가협회가 주최하고 과학기술정보통신부·한국인터넷진흥원 등이 후원하는 \'인터넷에코어워드(iEco Award) 2019\'와 \'소셜아이어워드(SOCIALl i-Award) 2019\'에서 각각 \'인터넷소통분야 대상\', \'시군구 지자체 페이스북 분야 최우수상\'을 수상했다고 4일 밝혔다."'

In [479]:
s = ["안녕","반가워"]

In [482]:
old_tokenizer(["test1","test2"],["test1","test2"])

{'input_ids': [[0, 87, 8119, 2083, 2, 87, 8119, 2083, 2], [0, 87, 8119, 2302, 2, 87, 8119, 2302, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [498]:
mask_pattern = new_tokenizer.mask_token

In [503]:
sentence = "그는 보통적으로 대한민국 국내에서는 1965년 영국·미국 합작 영화 《닥터 지바고(Doctor Zhivago)》의 OST 작품인 《》라는 영어 노래의 오리지널 원곡 가수로 잘 알려져 있으며 멕시코 여성 피아노 연주자 겸 작곡가 콘수엘로 벨라스케스(Consuelo Velzquez)가 작사 및 작곡하고 미국의 라틴 팝 보컬 음악 그룹 ""트리오 로스 판초스(Trio Los Panchos)""가 오리지널로 노래한 원곡인 《》라는 스페인어 노래를 리메이크로 재해석하여 히트한 가수로도 대한민국 국내에 널리 알려져 있다."

In [504]:

sentence = re.sub(r"\s+", " ", sentence).strip()
sentence= re.sub(r"\(\s?\)", mask_pattern, sentence)
sentence = re.sub(r"《\s?》", mask_pattern, sentence)
sentence

'그는 보통적으로 대한민국 국내에서는 1965년 영국·미국 합작 영화 《닥터 지바고(Doctor Zhivago)》의 OST 작품인 [MASK]라는 영어 노래의 오리지널 원곡 가수로 잘 알려져 있으며 멕시코 여성 피아노 연주자 겸 작곡가 콘수엘로 벨라스케스(Consuelo Velzquez)가 작사 및 작곡하고 미국의 라틴 팝 보컬 음악 그룹 트리오 로스 판초스(Trio Los Panchos)가 오리지널로 노래한 원곡인 [MASK]라는 스페인어 노래를 리메이크로 재해석하여 히트한 가수로도 대한민국 국내에 널리 알려져 있다.'

## 숫자 표현 (구둣점)

In [745]:
def num_punctuation_delete(sentence):
    m = re.search(r'[0-9]{1,3},[0-9]{3}(,[0-9]{3}){0,}',sentence)
    while( m !=None):
        origin = sentence[m.start():m.end()]
        replace_num = re.sub(',','',origin)
        sentence = sentence[:m.start()]+replace_num+sentence[m.end():]
        m = re.search(r'[0-9]{1,3},[0-9]{3}(,[0-9]{3}){0,}',sentence)
    return sentence

In [748]:
sentence = '"샤 자한이 죽기 직전인 1666년, 그는 지구 상에서 가장 영향력 강한 사람이었고, 무굴 제국은 3,000,000 km2에 달하는 영토를 지배했으며, 그의 수도는 세계에서 가장 크고 번영했으며, 아름다운 건축물들을 많이 가지고 있었다."'

In [749]:
num_punctuation_delete(sentence)

'"샤 자한이 죽기 직전인 1666년, 그는 지구 상에서 가장 영향력 강한 사람이었고, 무굴 제국은 3000000 km2에 달하는 영토를 지배했으며, 그의 수도는 세계에서 가장 크고 번영했으며, 아름다운 건축물들을 많이 가지고 있었다."'

In [777]:
def four_double_quotation_delete(sentence,tokenizer):
    if len(re.findall('""""', sentence)) == 1:
        sentence = re.sub('""""',tokenizer.mask_token,sentence)
    else:
        sentence = re.sub('""""', '"', sentence)
    return sentence

In [778]:
sentence='"스펜서는 1989년 뉴질랜드 태생의 호주 배우 러셀 크로우와 함께 영화 크로싱("""")에 남녀 주연으로 출연해 만남과 헤어짐을 반복했다."'

In [779]:
four_double_quotation_delete(sentence,new_tokenizer)

'"스펜서는 1989년 뉴질랜드 태생의 호주 배우 러셀 크로우와 함께 영화 크로싱([MASK])에 남녀 주연으로 출연해 만남과 헤어짐을 반복했다."'

In [780]:
sentence = '"이는 미국의 스페인어 방송을 통해 시청된 3번째로 가장 많은 시청자를 낸 경기로, 6월 27일 아르헨티나의 멕시코전 승리 (9, 405, 000) 와 2007년 12월 3일 방영된 텔레비전 드라마 """"사랑의 증류"""" (Destilando Amor, 901만 8천명)의 뒤를 이었다."'

In [781]:
four_double_quotation_delete(sentence,new_tokenizer)

'"이는 미국의 스페인어 방송을 통해 시청된 3번째로 가장 많은 시청자를 낸 경기로, 6월 27일 아르헨티나의 멕시코전 승리 (9, 405, 000) 와 2007년 12월 3일 방영된 텔레비전 드라마 "사랑의 증류" (Destilando Amor, 901만 8천명)의 뒤를 이었다."'

In [794]:
import re


# 따옴표 처리
## """" 제거  ("""")이 하나인 경우 : 그 안의 문장이 생략되었다 보고 마스크 토큰 삽입
## 다수인 경우 전부 "로 치환
def four_double_quotation_delete(sentence,tokenizer):
    if len(re.findall('""""', sentence)) == 1:
        sentence = re.sub('""""',tokenizer.mask_token,sentence)
    else:
        sentence = re.sub('""""', '"', sentence)
    return sentence


## "" -> "으로 변경
def double_quotation_to_quotation(sentence):
    sentence = re.sub(r'""', '"', sentence)
    return sentence


## 따옴표 계열 통일
def quotation_consist(sentence):
    sentence = re.sub(r"[‘’′＇`]", "'", sentence)
    sentence = re.sub(r"[“”]", '"', sentence)
    return sentence


# 괄호 처리
## 소괄호 처리
def parenthesis_consist(sentence):
    sentence = re.sub(r"（", "(", sentence)
    sentence = re.sub(r"）", ")", sentence)
    return sentence


## 브래킷 처리
def bracket_consist(sentence):
    sentence = re.sub(r"[«≪〈]", "《", sentence)
    sentence = re.sub(r"[»≫〉]", "》", sentence)
    return sentence


## 박스 처리
def box_consist(sentence):
    sentence = re.sub(r"[˹『｢]", "「", sentence)
    sentence = re.sub(r"[˼』｣]", "」", sentence)
    return sentence


# -처리
def bar_consist(sentence):
    sentence = re.sub(r"[–—―]", "-", sentence)
    return sentence


# cdot 처리
def cdot_consist(sentence):
    sentence = re.sub(r"[⋅･•・]", "·", sentence)
    return sentence


# 특수기호 제거
def symbol_delete(sentence):
    sentence = re.sub(r"[▲△▴▵□☎☏⁺∞Ⓐ®𑀫𑀕𑀥★☆♡♥※˘³𑀫𑀕𑀥↔]", " ", sentence)

    return sentence


# 야구에서 사용되는 분수표현 대체
def frac_consist(sentence):
    sentence = re.sub(r"⅓", " 1/3", sentence)
    sentence = re.sub(r"⅔", " 2/3", sentence)
    return sentence

# 숫자 구두점 제거  1,000 -> 1000
def num_punctuation_delete(sentence):
    m = re.search(r'[0-9]{1,3},[0-9]{3}(,[0-9]{3}){0,}',sentence)
    while( m !=None):
        origin = sentence[m.start():m.end()]
        replace_num = re.sub(',','',origin)
        sentence = sentence[:m.start()]+replace_num+sentence[m.end():]
        m = re.search(r'[0-9]{1,3},[0-9]{3}(,[0-9]{3}){0,}',sentence)
    return sentence

# 회사 표현 대체
def company_consist(sentence):
    sentence = re.sub(r"㈔", "(사)", sentence)
    sentence = re.sub(r"㈜", "(주)", sentence)
    return sentence


# 단위 기호 통일
def measure_consist(sentence):
    sentence = re.sub(r"㎏", "kg", sentence)
    sentence = re.sub(r"ℓ", "L", sentence)
    sentence = re.sub(r"㎖", "mL", sentence)
    sentence = re.sub(r"㎜", "mm", sentence)
    sentence = re.sub(r"㎞", "km", sentence)
    sentence = re.sub(r"㎡", "m²", sentence)
    sentence = re.sub(r"㎿", "MW", sentence)
    sentence = re.sub(r"ｍ", "m", sentence)
    sentence = re.sub(r"°", "도", sentence)

    return sentence


# 로마 숫자 -> 알파벳으로 변경
def roma_to_num(sentence):
    # https://unicode-table.com/kr/sets/roman-numerals/
    sentence = re.sub(r"Ⅰ", "I", sentence)
    sentence = re.sub(r"Ⅱ", "II", sentence)
    sentence = re.sub(r"Ⅲ", "III", sentence)
    sentence = re.sub(r"Ⅳ", "IV", sentence)
    sentence = re.sub(r"Ⅴ", "V", sentence)
    sentence = re.sub(r"Ⅵ", "VI", sentence)
    sentence = re.sub(r"Ⅶ", "VII", sentence)
    sentence = re.sub(r"Ⅷ", "VIII", sentence)
    sentence = re.sub(r"Ⅸ", "IX", sentence)
    sentence = re.sub(r"Ⅹ", "X", sentence)
    sentence = re.sub(r"Ⅺ", "XI", sentence)
    sentence = re.sub(r"Ⅻ", "XII", sentence)
    sentence = re.sub(r"Ⅼ", "L", sentence)
    sentence = re.sub(r"Ⅽ", "C", sentence)
    sentence = re.sub(r"Ⅾ", "D", sentence)
    sentence = re.sub(r"Ⅿ", "M", sentence)

    sentence = re.sub(r"ⅰ", "i", sentence)
    sentence = re.sub(r"ⅱ", "ii", sentence)
    sentence = re.sub(r"ⅲ", "iii", sentence)
    sentence = re.sub(r"ⅳ", "iv", sentence)
    sentence = re.sub(r"ⅴ", "v", sentence)
    sentence = re.sub(r"ⅵ", "vi", sentence)
    sentence = re.sub(r"ⅶ", "vii", sentence)
    sentence = re.sub(r"ⅷ", "viii", sentence)
    sentence = re.sub(r"ⅸ", "ix", sentence)
    sentence = re.sub(r"ⅹ", "x", sentence)
    sentence = re.sub(r"ⅺ", "xi", sentence)
    sentence = re.sub(r"ⅻ", "xii", sentence)
    sentence = re.sub(r"ⅼ", "l", sentence)
    sentence = re.sub(r"ⅽ", "c", sentence)
    sentence = re.sub(r"ⅾ", "d", sentence)
    sentence = re.sub(r"ⅿ", "m", sentence)

    return sentence


# 그외 유니코드 이슈 처리
def unicode_err_consist(sentence):
    sentence = re.sub(r"％", "%", sentence)
    sentence = re.sub(r"，", ",", sentence)
    sentence = re.sub(r"／", "/", sentence)
    sentence = re.sub(r"１", "1", sentence)
    sentence = re.sub(r"：", ":", sentence)
    sentence = re.sub(r"？", "?", sentence)
    sentence = re.sub(r"～", "~", sentence)

    return sentence


# 전체 텍스트 전처리
def text_preprocessing(sentence, tokenizer):
    # 따옴표 계열 처리 과정
    sentence = four_double_quotation_delete(sentence,tokenizer)
    sentence = double_quotation_to_quotation(sentence)
    sentence = quotation_consist(sentence)

    # 괄호 계열 처리
    sentence = parenthesis_consist(sentence)
    sentence = bracket_consist(sentence)
    sentence = box_consist(sentence)

    # bar 처리
    sentence = bar_consist(sentence)

    # cdot 처리
    sentence = cdot_consist(sentence)

    # 특수 기호 제거
    sentence = symbol_delete(sentence)
    
    # 숫자 구두점 제거
    sentence = num_punctuation_delete(sentence)
    
    # 분수 표현(야구) 통일
    sentence = frac_consist(sentence)

    # 회사표현 통일
    sentence = company_consist(sentence)

    # 단위표현 통일
    sentence = measure_consist(sentence)

    # 로마숫자 숫자로 변경
    sentence = roma_to_num(sentence)

    # 그외 유니코드 이슈 처리
    sentence = unicode_err_consist(sentence)

    # 공백 두번 이상인 것 처리 및 앞 뒤 공백 제거
    sentence = re.sub(r"\s+", " ", sentence).strip()

    # 빈괄호 마스크 처리 (의미상 제거되면 안됨)
    mask_pattern = tokenizer.mask_token
    sentence = re.sub(r"\(\s?\)", mask_pattern, sentence)
    sentence = re.sub(r"《\s?》", mask_pattern, sentence)

    return sentence


In [797]:
text_preprocessing(sentence,new_tokenizer)

'"2011년 1월 26일, 바르찰리는 VfL 볼프스부르크를 떠나 세리에 A의 거함 유벤투스 FC와 €300000 고정과 €300000 옵션의 계약을 맺으며 이탈리아 무대로 복귀하였다."'

In [509]:
sentence = '"러시아측으로는 잉글랜드(1707년에 영국), 하노버, 네덜란드, 프로이센, 폴란드-리투아니아 연합(1701년-1704년, 1704년-1709년, 1709년-1719년)이 참여했고, 스웨덴 측으로는 폴란드-리투아니아 연합(1704년-1709년), 오스만 제국과 카자크 수장국이 참여하였다."'

In [784]:
sentence = '"국제 농구 연맹은 1932년 스위스 제네바에서 ""국제 아마추어 농구 연맹""("""")이라는 명칭으로 창설되었다."'

In [786]:
sentence = '"에피소드에서는 외계 시간여행자인 닥터 (크리스토퍼 에클스턴)이 새로운 동행자 로즈 타일러 (빌리 파이퍼)를 타디스를 이용한 첫번째 시공간 여행으로 5,000,000년에 데려간다."'

In [790]:
sentence = '대한민국에서는 2008년 대원미디어㈜에서 수입하여 자사인 대원방송㈜에서 더빙 작업을 진행하여 《짱구는 못말려 극장판: 부리부리 3분 대작전》이라는 이름으로 방영되었다.'

In [792]:
sentence = '"(KIA 내야수 그, 투수 이동현, 외야수 김경언 ↔ 한화 투수 안영명, 박성호, 외야수 김다원). 이 트레이드 상대 중 안영명은 이듬해 이범호의 보상 선수로 팀에 복귀했다."'

In [796]:
sentence = '"2011년 1월 26일, 바르찰리는 VfL 볼프스부르크를 떠나 세리에 A의 거함 유벤투스 FC와 €300,000 고정과 €300,000 옵션의 계약을 맺으며 이탈리아 무대로 복귀하였다."'